
#### Description of QueryProcessor


##### Description of QueryProcessor


##### Features

- Dynamic table_changes for tables with version_param.
- Optional per-table filter applied automatically.
- Multi-column joins handled correctly.
- Union across multiple tables.
- Dynamic catalog replacements like ${entity} supported.
- Runtime version map allows flexible filtering of selected tables.

---

##### 1. Function: renderSqlTemplate,  renderSqlTemplateExtended

This function can be used to check the the generated SQL query before actual test, we can copy the query to SQL editor to run to validate first 

```scala


// Basic function: read master table by start/end version and read appendix tables as end version
def renderSqlTemplate(
  sqlTemplateConfig: String, 
  tableVersionMap: Map[String, (Long, Long)] = Map.empty
): Unit = { ... }



// Extended function(experimental): extend the logic to return the row of the latest version if multiple rows exist
def renderSqlTemplateExtended(
  sqlTemplateConfig: String, 
  tableVersionMap: Map[String, (Long, Long)] = Map.empty
): Unit = { ... }


```

**Paramters**:
- sqlTemplateConfig: The SQL-query configuration that defines the SQL query. It contains placeholders(catalog) that need to be replaced.
- tableVersionMap: A map for rom upstream tables: tableName → (startVersion, endVersion), where `startVersion` and `endVersion` define the inclusive version range to process for each upstream table.  the map data can be created using Watermarks

**tableVersionMap Example**
```scala
val versionMap = Map(
  "ag_content_ims_acs_prod.gold_entity.d_spmaster" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.d_orgmaster" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.spmaster_publication_link" -> (1L, 2L)
)

```

---

##### 2. Function: runSqlAndSave

This function is used to actually create the delta Paffected K table speficied in SQL config and save data to the table

```scala


def runSqlAndSave(
  sqlTemplateConfig: String,
  tableVersionMap: Map[String, (Long, Long)] = Map.empty,
  targetTableName: String = "",
  dryRun: Boolean = false
): Unit = { ... }

```

**Paramters**:
- sqlTemplateConfig: The SQL-query configuration that defines the SQL query. It contains placeholders(catalog) that need to be replaced.
- tableVersionMap: A map for upstream tables: tableName → (startVersion, endVersion), where `startVersion` and `endVersion` define the inclusive version range to process for each upstream table.  the map data can be created using Watermarks 
- targetTableName: The taget delta tbale to save the output Dataframe
- dryRun: Save data to the delta table when dryRun is False


---


##### 3. Scheam Map Definition: SchemaResolver.SCHEMA_MAP

This object warp the information on catalog, schema, environment and  version varibable management, which is used to render the SQl query with the dynamic varibale for catalog.


```scala
val catalogMap = SchemaResolver.SCHEMA_MAP
```


**SCHEMA_MAP Example**

```scala

 Map(
  pprn -> ag_content_ims_acs_prod.gold_pprn
  dap_metrics_pprn -> ag_ra_search_analytics_data_dev.dap_metrics_pprn_v1_0
  dap_entity_wos -> ag_ra_search_analytics_data_dev.dap_entity_wos_v1_0
  dap_metrics_wos -> ag_ra_search_analytics_data_dev.dap_metrics_wos_v1_0
  dap_entity_enrich -> ag_ra_search_analytics_data_dev.dap_entity_enrich_v1_0
  dap_reference -> ag_ra_search_analytics_data_dev.dap_reference_v1_0
)

```

---








#### object QueryProcessor

##### 1. Setup Param 

In [0]:
// the following parameters are from thebundle's databrick yaml config file 

//  pass the parameters
dbutils.widgets.text("source_catalog", "ag_content_ims_acs")
dbutils.widgets.text("source_environment", "prod")
dbutils.widgets.text("source_version", "v1_0_0")

dbutils.widgets.text("target_catalog", "ag_ra_search_analytics_data")
dbutils.widgets.text("target_environment", "dev")
dbutils.widgets.text("target_version", "v1_0")

// dynamic paramters
val source_catalog = dbutils.widgets.get("source_catalog")
val source_environment = dbutils.widgets.get("source_environment")
val source_version = dbutils.widgets.get("source_version")
val target_catalog = dbutils.widgets.get("target_catalog")
val target_environment = dbutils.widgets.get("target_environment")
val target_version = dbutils.widgets.get("target_version")


source_catalog: String = "ag_content_ims_acs"
source_environment: String = "prod"
source_version: String = ""
target_catalog: String = "ag_ra_search_analytics_data"
target_environment: String = "dev"
target_version: String = "v1_0"

##### 2. QueryProcessor

In [0]:

object SchemaResolver {

  private def getWidget(name: String, default: String): String = {
    try {
      val value = dbutils.widgets.get(name)
      if (value == null || value.isEmpty) default else value
    } catch {
      case _: Throwable => default
    }
  }

  private val source_catalog = getWidget("source_catalog", "ag_content_ims_acs")
  private val source_environment = getWidget("source_environment", "prod")
  private val source_version = getWidget("source_version", "")

  private val target_catalog = getWidget("target_catalog", "ag_ra_search_analytics_data")
  private val target_environment = getWidget("target_environment", "dev")
  private val target_version = getWidget("target_version", "v1_0")

  private val dapSchemaBases = Seq(
      "dap_entity_wos",
      "dap_metrics_wos",
      "dap_entity_pprn",
      "dap_metrics_pprn",
      "dap_docs",
      "dap_reference",
      "dap_sort_ref",
      "dap_entity_enrich",
      "dap_grant",
      "dap_prod_core",
      "dap_ops",
      "dap_work"
  )

  private val acsSchemaBases= Seq(
      "gold_entity",
      "gold_wos",
      "gold_pprn"
  )

  val SCHEMA_MAP: Map[String, String] = {
    val srcBase = s"${source_catalog}_${source_environment}"
    val tgtBase = s"${target_catalog}_${target_environment}"
    val versionSuffix = if (source_version.isEmpty) "" else s"_${source_version}"
    val tgtVerSuffix = s"_${target_version}"

    // ACS schemas: key → base name (without version)
    val acsSchemas = Seq(
      "entity" -> s"$srcBase.gold_entity$versionSuffix",
      "wos"    -> s"$srcBase.gold_wos$versionSuffix",
      "pprn"   -> s"$srcBase.gold_pprn$versionSuffix"
    )

    // DAP schemas: key → base name (without target version suffix)
    val dapSchemas: Seq[(String, String)] =
      dapSchemaBases.map { key =>
        key -> s"$tgtBase.$key$tgtVerSuffix"
      }

    // Optional sandbox entry
    val sandboxEntry = Seq(
      "dap" -> s"$tgtBase.sandbox$tgtVerSuffix"
    )
    // Combine all
    (acsSchemas ++ dapSchemas ++ sandboxEntry).toMap
  }

  val SCHEMAS: Seq[String] = {
    val srcBase = s"${source_catalog}_${source_environment}"
    val tgtBase = s"${target_catalog}_${target_environment}"
    val versionSuffix = if (source_version.isEmpty) "" else s"_${source_version}"
    val tgtVerSuffix  = s"_${target_version}"

    // ACS schema names
    val acsSchemas = acsSchemaBases.map(name => s"$srcBase.$name$versionSuffix")

    // DAP schema names
    val dapSchemas = dapSchemaBases.map(baseName => s"$tgtBase.$baseName$tgtVerSuffix")

    acsSchemas ++ dapSchemas
  }
}



defined object SchemaResolver

In [0]:
import org.yaml.snakeyaml.Yaml
import scala.jdk.CollectionConverters._
import org.apache.spark.sql.DataFrame

// ---------------------- Case Classes ----------------------
case class TableJoin(
  table_name: String,
  alias: String,
  pk: String,
  fk: Option[String] = None,
  versionParam: Option[String] = None,  // dynamic version key
  filter: Option[String] = None         // optional WHERE filter
)

case class UnionTable(
  select_columns: String,
  joins: Seq[TableJoin]
)

case class OutputTable(name: String)

case class UnionConfig(
  output_table: OutputTable,
  union_tables: Seq[UnionTable]
)

object QueryProcessor {

  private def cleanTableName(name: String): String =
    name.replace("\"", "").trim

  // ------------------ YAML Parser ------------------
  private def parseQueryConfigFromYaml(
      queryConfigYaml: String
    ): UnionConfig = {
    var replacedYaml = queryConfigYaml
    SchemaResolver.SCHEMA_MAP.foreach { case (key, value) =>
      replacedYaml = replacedYaml.replace(s"$${$key}", value)
    }

    val yaml = new Yaml()
    val obj = yaml.load(replacedYaml).asInstanceOf[java.util.Map[String, Object]]

    val outputTableMap = obj.get("output_table").asInstanceOf[java.util.Map[String, Object]]
    val outputTable = OutputTable(outputTableMap.get("name").toString)

    val unionTablesJava = obj.get("union_tables")
      .asInstanceOf[java.util.List[java.util.Map[String, Object]]]

    val unionTables = unionTablesJava.asScala.map { ut =>
      val joinsJava = ut.get("joins")
        .asInstanceOf[java.util.List[java.util.Map[String, Object]]]

      UnionTable(
        select_columns = ut.get("select_columns").toString,
        joins = joinsJava.asScala.map { j =>
          TableJoin(
            table_name = cleanTableName(j.get("table_name").toString),
            alias = j.get("alias").toString,
            pk = j.get("pk").toString,
            fk = Option(j.get("fk")).map(_.toString),
            versionParam = Option(j.get("version_param")).map(_.toString),
            filter = Option(j.get("filter")).map(_.toString)
          )
        }.toSeq
      )
    }

    UnionConfig(outputTable, unionTables.toSeq)
  }

  // ------------------ Query builder ------------------
  private def buildJoinQuery(
      tables: Seq[TableJoin],
      selectColumns: String,
      tableVersionMap: Map[String, (Long, Long)]
    ): String = {

    val base = tables.head

    val invalidVersion = tables.exists { t =>
      t.versionParam.map(_.toString)
      .flatMap(tableVersionMap.get) match {
        case Some((start, _)) if start == -1 => true
        case _ => false
      }
    }
    if (invalidVersion) return ""
    
    def tableRef(t: TableJoin): String = {
      t.versionParam.flatMap(tableVersionMap.get) match {
        case Some((start, end)) =>
          s"table_changes('${t.table_name}', $start, $end) ${t.alias}"
        case None =>
         Option(t.table_name).map(_.toString).flatMap(tableVersionMap.get) match {
          case Some((start, end)) =>
           s"${t.table_name} VERSION AS OF $end ${t.alias}"
          case None =>
            s"${t.table_name} ${t.alias}"
         }
      }
    } 

    val joins = tables.tail.map { t =>
      val pkCols = t.pk.split(",").map(_.trim)
      val fkCols = t.fk.map(_.split(",").map(_.trim)).getOrElse(Array.empty)

      val cond =
        if (fkCols.isEmpty) ""
        else pkCols.zip(fkCols).map { case (pk, fk) => s"$pk = $fk" }.mkString(" AND ")

      s"JOIN ${tableRef(t)} ON $cond"
    }.mkString("\n").trim

    //val where = tables.flatMap(_.filter).mkString("WHERE\n  ", "\n  AND ", "")  
    val filters: Seq[String] = tables
      .flatMap(t => t.filter.map(_.trim)) // trim inside the Option and flatten
      .filter(_.nonEmpty)                 // remove empty strings

    val where =
      if (filters.isEmpty)
        ""
      else
        filters.mkString("WHERE\n  ", "\n  AND ", "")




    s"""
      |SELECT $selectColumns
      |FROM ${tableRef(base)}
      |$joins
      |$where
      |""".stripMargin.trim
  }

  private def buildUnionQuery(
      templateConfig: UnionConfig,
      versionMap: Map[String, (Long, Long)]
    ): String = {
    val queries = templateConfig.union_tables
      .map { ut =>
        buildJoinQuery(ut.joins, ut.select_columns, versionMap)
      }
      .filter(_.nonEmpty)

    if (queries.isEmpty) {
      "SELECT NULL WHERE 1 = 0"   // dummy query when nothing to union
    } else {
      queries.mkString("\nUNION\n")
    }
  }

  private def buildUnionQueryExtended(
      templateConfig: UnionConfig,
      versionMap: Map[String, (Long, Long)]
    ): String = {
    val queries = templateConfig.union_tables
      .map { ut =>
        buildJoinQuery(ut.joins, ut.select_columns, versionMap)
      }
      .filter(_.nonEmpty)

    if (queries.isEmpty) {
      "SELECT NULL WHERE 1 = 0"   // dummy query when nothing to union
    } else {
      queries.zipWithIndex
      .map { case (q, i) =>
        s"SELECT * FROM (WITH tbl_$i AS ($q) SELECT * FROM tbl_$i WHERE rn = 1)"
      }
      .mkString("\nUNION\n")
     // queries.map { t => s"SELECT * FROM ( WITH final AS ( $t ) SELECT * FROM final WHERE rn = 1 )"
     // }.mkString("\nUNION\n")
    }
  }

  // ------------------ Query Render ------------------

  def renderSqlTemplate(
      sqlTemplateConfig: String,
      tableVersionMap: Map[String, (Long, Long)] = Map.empty
    ): String = {

    val sqlConfig = parseQueryConfigFromYaml(sqlTemplateConfig)
    
    buildUnionQuery(sqlConfig, tableVersionMap)
  }

  def renderSqlTemplateExtended(
      sqlTemplateConfig: String,
      tableVersionMap: Map[String, (Long, Long)] = Map.empty
    ): String = {

    val sqlConfig = parseQueryConfigFromYaml(sqlTemplateConfig)

    buildUnionQueryExtended(sqlConfig, tableVersionMap)
  }

  // ------------------ Run Query  ------------------
  def runSqlAndSave(
      sqlTemplateConfig: String,
      tableVersionMap: Map[String, (Long, Long)] = Map.empty,
      targetTableName: String = "",
      dryRun: Boolean = false
    ): Unit = {

    val sqlConfig = parseQueryConfigFromYaml(sqlTemplateConfig)
    val sqlQuery = buildUnionQuery(sqlConfig, tableVersionMap)

    val tgtTableName = Option(targetTableName).getOrElse(sqlConfig.output_table.name)

    println("Generated SQL Query:\n" + sqlQuery)

    val resultDf: DataFrame = spark.sql(sqlQuery)
    
    // Save as Delta table if needed
    println(s"Saving result to Delta table: $tgtTableName")
    if(!dryRun)
      resultDf.write.format("delta").mode("overwrite").saveAsTable(tgtTableName)
    else
      display(resultDf) 
  }
}


import org.yaml.snakeyaml.Yaml

import scala.jdk.CollectionConverters._

import org.apache.spark.sql.DataFrame

// ---------------------- Case Classes ----------------------

defined class TableJoin
defined class UnionTable
defined class OutputTable
defined class UnionConfig
defined object QueryProcessor

##### 3. Test QueryProcessor - Example-1

In [0]:
SchemaResolver.SCHEMA_MAP.foreach{ case (k,v) => println(s"$k -> $v") }
println("------")
SchemaResolver.SCHEMAS.foreach{ schema => println(schema) }  

pprn -> ag_content_ims_acs_prod.gold_pprn
dap_metrics_pprn -> ag_ra_search_analytics_data_dev.dap_metrics_pprn_v1_0
dap_entity_wos -> ag_ra_search_analytics_data_dev.dap_entity_wos_v1_0
dap_metrics_wos -> ag_ra_search_analytics_data_dev.dap_metrics_wos_v1_0
dap_entity_enrich -> ag_ra_search_analytics_data_dev.dap_entity_enrich_v1_0
dap_reference -> ag_ra_search_analytics_data_dev.dap_reference_v1_0
entity -> ag_content_ims_acs_prod.gold_entity
dap_prod_core -> ag_ra_search_analytics_data_dev.dap_prod_core_v1_0
dap_ops -> ag_ra_search_analytics_data_dev.dap_ops_v1_0
dap_grant -> ag_ra_search_analytics_data_dev.dap_grant_v1_0
dap_docs -> ag_ra_search_analytics_data_dev.dap_docs_v1_0
dap_entity_pprn -> ag_ra_search_analytics_data_dev.dap_entity_pprn_v1_0
wos -> ag_content_ims_acs_prod.gold_wos
dap -> ag_ra_search_analytics_data_dev.sandbox_v1_0
dap_work -> ag_ra_search_analytics_data_dev.dap_work_v1_0
dap_sort_ref -> ag_ra_search_analytics_data_dev.dap_sort_ref_v1_0
------
ag_content_ims_

In [0]:



val queryAlma: String =  """
output_table:
  name: "${dap}.pk_alma_affected"

union_tables:
  - select_columns: "journal_acs_key AS journal_key"
    joins:
      - table_name: "${wos}.journal_acs_publication_link"
        alias: "jlink"
        pk: "journal_acs_key"
        version_param: ${wos}.journal_acs_publication_link   # optional: will use dynamic version
        filter: "jlink._change_type IN ('insert',  'update_postimage')"  # optional

  - select_columns: "journal_key AS journal_key"
    joins:
      - table_name: "${entity}.d_alma_subscriptions"
        alias: "alma"
        pk: "journal_key"
        filter: "alma.__end_at IS NULL"

  - select_columns: "jlink.journal_acs_key AS journal_key"
    joins:
      - table_name: "${wos}.publisher_publication_link"
        alias: "plink"
        pk: "uid"
      - table_name: "${wos}.journal_acs_publication_link"
        alias: "jlink"
        pk: "journal_acs_key"
        fk: "plink.uid = jlink.uid"
        filter: "plink.__end_at IS NULL"


"""

queryAlma: String = """
output_table:
  name: "${dap}.pk_alma_affected"

union_tables:
  - select_columns: "journal_acs_key AS journal_key"
    joins:
      - table_name: "${wos}.journal_acs_publication_link"
        alias: "jlink"
        pk: "journal_acs_key"
        version_param: ${wos}.journal_acs_publication_link   # optional: will use dynamic version
        filter: "jlink._change_type IN ('insert',  'update_postimage')"  # optional

  - select_columns: "journal_key AS journal_key"
    joins:
      - table_name: "${entity}.d_alma_subscriptions"
        alias: "alma"
        pk: "journal_key"
        filter: "alma.__end_at IS NULL"

  - select_columns: "jlink.journal_acs_key AS journal_key"
    joins:
      - table_name: "${wos}.publisher_publication_link"
        alias: "plink"
        pk: "uid"
      - table_name: "${wos}.journal_acs_publication_link"
        alias: "jlink"
        pk: "journal_acs_key"
        fk: "plink.uid = jlink.uid"
        filter: "plink.__end_at IS NULL

In [0]:
val versionMap = Map(
  "ag_content_ims_acs_prod.gold_wos.journal_acs_publication_link" -> (1L, 2L)
)

println(QueryProcessor.renderSqlTemplate(queryAlma, versionMap).stripMargin)

SELECT journal_acs_key AS journal_key
FROM table_changes('ag_content_ims_acs_prod.gold_wos.journal_acs_publication_link', 1, 2) jlink

WHERE
  jlink._change_type IN ('insert',  'update_postimage')
UNION
SELECT journal_key AS journal_key
FROM ag_content_ims_acs_prod.gold_entity.d_alma_subscriptions alma

WHERE
  alma.__end_at IS NULL
UNION
SELECT jlink.journal_acs_key AS journal_key
FROM ag_content_ims_acs_prod.gold_wos.publisher_publication_link plink
JOIN ag_content_ims_acs_prod.gold_wos.journal_acs_publication_link VERSION AS OF 2 jlink ON journal_acs_key = plink.uid = jlink.uid
WHERE
  plink.__end_at IS NULL


versionMap: Map[String, (Long, Long)] = Map(
  "ag_content_ims_acs_prod.gold_wos.journal_acs_publication_link" -> (1L, 2L)
)

##### 4. Test QueryProcessor- Example-2

In [0]:
val queryAP: String =  """
  output_table:
    name: "${dap}.pk_affected_sp"
  union_tables:
    - select_columns: "sp_id AS sp_id, sp._change_type as _change_type, sp.__end_at"
      joins:
        - table_name: ${entity}.d_spmaster
          alias: "sp"
          pk: "sp_id"
          version_param: ${entity}.d_spmaster   # optional: will use dynamic version
          filter: "sp._change_type IN ('insert',  'update_postimage') AND sp.__end_at IS NULL"  # optional
          
    - select_columns: "spl.sp_id AS sp_id,  org.update_postimage AS _change_type, org.__end_at"
      joins:
        # Base table: updated orgmaster
        - table_name: ${entity}.d_orgmaster
          alias: org
          pk: "org.org_pguid"
          version_param: ${entity}.d_orgmaster 
          filter: "org._change_type IN ('update_postimage') AND org.__end_at IS NULL"

        # Join to orgmaster_publication_link (by org_pguid)
        - table_name: ${entity}.orgmaster_publication_link
          alias: oml
          pk: "oml.org_pguid"
          fk: "org.org_pguid"
          filter: "oml.__end_at IS NULL"

        # Join to spmaster_publication_link (by uid AND author_position)
        - table_name: ${entity}.spmaster_publication_link
          alias: spl
          pk: "spl.uid, spl.author_position"
          fk: "oml.uid, oml.address_position"
          filter: "spl.__end_at IS NULL"

"""

queryAP: String = """
  output_table:
    name: "${dap}.pk_affected_sp"
  union_tables:
    - select_columns: "sp_id AS sp_id, sp._change_type as _change_type, sp.__end_at"
      joins:
        - table_name: ${entity}.d_spmaster
          alias: "sp"
          pk: "sp_id"
          version_param: ${entity}.d_spmaster   # optional: will use dynamic version
          filter: "sp._change_type IN ('insert',  'update_postimage') AND sp.__end_at IS NULL"  # optional
          
    - select_columns: "spl.sp_id AS sp_id,  org.update_postimage AS _change_type, org.__end_at"
      joins:
        # Base table: updated orgmaster
        - table_name: ${entity}.d_orgmaster
          alias: org
          pk: "org.org_pguid"
          version_param: ${entity}.d_orgmaster 
          filter: "org._change_type IN ('update_postimage') AND org.__end_at IS NULL"

        # Join to orgmaster_publication_link (by org_pguid)
        - table_name: ${entity}.orgmaster_publication_link
          alias: oml
     

In [0]:

val versionMap = Map(
  "ag_content_ims_acs_prod.gold_entity.d_spmaster" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.d_orgmaster" -> (1L, 4L),
  "ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link" -> (1L, 3L),
  "ag_content_ims_acs_prod.gold_entity.spmaster_publication_link" -> (1L, 5L)
)

println(QueryProcessor.renderSqlTemplate(queryAP, versionMap).stripMargin)




SELECT sp_id AS sp_id, sp._change_type as _change_type, sp.__end_at
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_spmaster', 1, 2) sp

WHERE
  sp._change_type IN ('insert',  'update_postimage') AND sp.__end_at IS NULL
UNION
SELECT spl.sp_id AS sp_id,  org.update_postimage AS _change_type, org.__end_at
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_orgmaster', 1, 4) org
JOIN ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link VERSION AS OF 3 oml ON oml.org_pguid = org.org_pguid
JOIN ag_content_ims_acs_prod.gold_entity.spmaster_publication_link VERSION AS OF 5 spl ON spl.uid = oml.uid AND spl.author_position = oml.address_position
WHERE
  org._change_type IN ('update_postimage') AND org.__end_at IS NULL
  AND oml.__end_at IS NULL
  AND spl.__end_at IS NULL


versionMap: Map[String, (Long, Long)] = Map(
  "ag_content_ims_acs_prod.gold_entity.d_spmaster" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.d_orgmaster" -> (1L, 4L),
  "ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link" -> (1L, 3L),
  "ag_content_ims_acs_prod.gold_entity.spmaster_publication_link" -> (1L, 5L)
)

In [0]:

val versionMap = Map(
  "ag_content_ims_acs_prod.gold_entity.d_spmaster" -> (-1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.d_orgmaster" -> (1L, 4L),
  "ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link" -> (1L, 3L),
  "ag_content_ims_acs_prod.gold_entity.spmaster_publication_link" -> (1L, 5L)
)

println(QueryProcessor.renderSqlTemplate(queryAP, versionMap).stripMargin)



SELECT spl.sp_id AS sp_id,  org.update_postimage AS _change_type, org.__end_at
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_orgmaster', 1, 4) org
JOIN ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link VERSION AS OF 3 oml ON oml.org_pguid = org.org_pguid
JOIN ag_content_ims_acs_prod.gold_entity.spmaster_publication_link VERSION AS OF 5 spl ON spl.uid = oml.uid AND spl.author_position = oml.address_position
WHERE
  org._change_type IN ('update_postimage') AND org.__end_at IS NULL
  AND oml.__end_at IS NULL
  AND spl.__end_at IS NULL


versionMap: Map[String, (Long, Long)] = Map(
  "ag_content_ims_acs_prod.gold_entity.d_spmaster" -> (-1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.d_orgmaster" -> (1L, 4L),
  "ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link" -> (1L, 3L),
  "ag_content_ims_acs_prod.gold_entity.spmaster_publication_link" -> (1L, 5L)
)

In [0]:

val versionMap = Map(
  "ag_content_ims_acs_prod.gold_entity.d_spmaster" -> (-1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.d_orgmaster" -> (-1L, 4L),
  "ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link" -> (1L, 3L),
  "ag_content_ims_acs_prod.gold_entity.spmaster_publication_link" -> (1L, 5L)
)

println(QueryProcessor.renderSqlTemplate(queryAP, versionMap).stripMargin)

SELECT NULL WHERE 1 = 0


versionMap: Map[String, (Long, Long)] = Map(
  "ag_content_ims_acs_prod.gold_entity.d_spmaster" -> (-1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.d_orgmaster" -> (-1L, 4L),
  "ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link" -> (1L, 3L),
  "ag_content_ims_acs_prod.gold_entity.spmaster_publication_link" -> (1L, 5L)
)

In [0]:
val queryAP: String =  """
  output_table:
    name: "${dap}.pk_affected_sp"
  union_tables:
    - select_columns: "sp_id AS sp_id, sp._change_type as _change_type, sp.__end_at, sp._commit_version, ROW_NUMBER() OVER ( PARTITION BY sp.sp_id ORDER BY sp._commit_version DESC) AS rn"
      joins:
        - table_name: ${entity}.d_spmaster
          alias: "sp"
          pk: "sp_id"
          version_param: ${entity}.d_spmaster   # optional: will use dynamic version
          filter: "sp._change_type IN ('insert',  'update_postimage') AND sp.__end_at IS NULL"  # optional
          
    - select_columns: "spl.sp_id AS sp_id,  'update_postimage' AS _change_type, org.__end_at, org._commit_version, ROW_NUMBER() OVER ( PARTITION BY org.org_pguid  ORDER BY org._commit_version DESC) AS rn"
      joins:
        # Base table: updated orgmaster
        - table_name: ${entity}.d_orgmaster
          alias: org
          pk: "org.org_pguid"
          version_param: ${entity}.d_orgmaster 
          filter: "org._change_type IN ('update_postimage') AND org.__end_at IS NULL"

        # Join to orgmaster_publication_link (by org_pguid)
        - table_name: ${entity}.orgmaster_publication_link
          alias: oml
          pk: "oml.org_pguid"
          fk: "org.org_pguid"
          filter: "oml.__end_at IS NULL"

        # Join to spmaster_publication_link (by uid AND author_position)
        - table_name: ${entity}.spmaster_publication_link
          alias: spl
          pk: "spl.uid, spl.author_position"
          fk: "oml.uid, oml.address_position"
          filter: "spl.__end_at IS NULL"

"""

queryAP: String = """
  output_table:
    name: "${dap}.pk_affected_sp"
  union_tables:
    - select_columns: "sp_id AS sp_id, sp._change_type as _change_type, sp.__end_at, sp._commit_version, ROW_NUMBER() OVER ( PARTITION BY sp.sp_id ORDER BY sp._commit_version DESC) AS rn"
      joins:
        - table_name: ${entity}.d_spmaster
          alias: "sp"
          pk: "sp_id"
          version_param: ${entity}.d_spmaster   # optional: will use dynamic version
          filter: "sp._change_type IN ('insert',  'update_postimage') AND sp.__end_at IS NULL"  # optional
          
    - select_columns: "spl.sp_id AS sp_id,  'update_postimage' AS _change_type, org.__end_at, org._commit_version, ROW_NUMBER() OVER ( PARTITION BY org.org_pguid  ORDER BY org._commit_version DESC) AS rn"
      joins:
        # Base table: updated orgmaster
        - table_name: ${entity}.d_orgmaster
          alias: org
          pk: "org.org_pguid"
          version_param: ${entity}.d_orgmaster 
          filter: "o

In [0]:

val versionMap = Map(
  "ag_content_ims_acs_prod.gold_entity.d_spmaster" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.d_orgmaster" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.spmaster_publication_link" -> (1L, 2L)
)

println(QueryProcessor.renderSqlTemplateExtended(queryAP, versionMap).stripMargin)


SELECT * FROM (WITH tbl_0 AS (SELECT sp_id AS sp_id, sp._change_type as _change_type, sp.__end_at, sp._commit_version, ROW_NUMBER() OVER ( PARTITION BY sp.sp_id ORDER BY sp._commit_version DESC) AS rn
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_spmaster', 1, 2) sp

WHERE
  sp._change_type IN ('insert',  'update_postimage') AND sp.__end_at IS NULL) SELECT * FROM tbl_0 WHERE rn = 1)
UNION
SELECT * FROM (WITH tbl_1 AS (SELECT spl.sp_id AS sp_id,  'update_postimage' AS _change_type, org.__end_at, org._commit_version, ROW_NUMBER() OVER ( PARTITION BY org.org_pguid  ORDER BY org._commit_version DESC) AS rn
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_orgmaster', 1, 2) org
JOIN ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link VERSION AS OF 2 oml ON oml.org_pguid = org.org_pguid
JOIN ag_content_ims_acs_prod.gold_entity.spmaster_publication_link VERSION AS OF 2 spl ON spl.uid = oml.uid AND spl.author_position = oml.address_position
WHERE
  org._change_ty

versionMap: Map[String, (Long, Long)] = Map(
  "ag_content_ims_acs_prod.gold_entity.d_spmaster" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.d_orgmaster" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link" -> (1L, 2L),
  "ag_content_ims_acs_prod.gold_entity.spmaster_publication_link" -> (1L, 2L)
)

In [0]:

// Save PK data to catalog table speficied in config
// dryRun = true -  No save operation
QueryProcessor.runSqlAndSave(queryAP, versionMap, "",  true)



Generated SQL Query:
SELECT sp_id AS sp_id, sp._change_type as _change_type, sp.__end_at, sp._commit_version, ROW_NUMBER() OVER ( PARTITION BY sp.sp_id ORDER BY sp._commit_version DESC) AS rn
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_spmaster', 1, 2) sp

WHERE
  sp._change_type IN ('insert',  'update_postimage') AND sp.__end_at IS NULL
UNION
SELECT spl.sp_id AS sp_id,  'update_postimage' AS _change_type, org.__end_at, org._commit_version, ROW_NUMBER() OVER ( PARTITION BY org.org_pguid  ORDER BY org._commit_version DESC) AS rn
FROM table_changes('ag_content_ims_acs_prod.gold_entity.d_orgmaster', 1, 2) org
JOIN ag_content_ims_acs_prod.gold_entity.orgmaster_publication_link VERSION AS OF 2 oml ON oml.org_pguid = org.org_pguid
JOIN ag_content_ims_acs_prod.gold_entity.spmaster_publication_link VERSION AS OF 2 spl ON spl.uid = oml.uid AND spl.author_position = oml.address_position
WHERE
  org._change_type IN ('update_postimage') AND org.__end_at IS NULL
  AND oml.__end_at IS 

sp_id,_change_type,__end_at,_commit_version,rn
urn:spm:10000894,insert,null,2,1
urn:spm:10001938,insert,null,2,1
urn:spm:10003890,insert,null,2,1
urn:spm:10004621,insert,null,2,1
urn:spm:10012671,insert,null,2,1
urn:spm:10014407,insert,null,2,1
urn:spm:1001577,insert,null,2,1
urn:spm:1001873,insert,null,2,1
urn:spm:10020265,insert,null,2,1
urn:spm:10022385,insert,null,2,1
